## Create Mike's dictionary of Morton codes with start and end points for the 512^3

In [1]:
desired_cube_side = 512
chunk_size = 64
array_cube_side = 2048

raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'

dest_folder_name = "sabl2048b" # B is the high-rate data
write_type = "prod" # or "back"
timestep_nr = 1
# timestep_range = range(1) # Ned's new High-rate fixed-dt has only 5 timesteps

In [10]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code


In [11]:
from utils import write_utils

import xarray as xr
# from utils import write_utils
import os

### Get target Folder list

<font color="orange">Don't delete the CD cell!</font>

In [12]:
%cd /home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt

/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt


In [13]:
merged_velocity = xr.open_dataset(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")

In [14]:
# Split 2048^3 into smaller 512^3 arrays
dims = [dim for dim in merged_velocity.dims]
dims.reverse() # use (nnz, nny, nnx) instead of (nnx, nny, nnz)

smaller_groups, range_list = write_utils.split_zarr_group(merged_velocity, desired_cube_side, dims)

In [15]:
# Given up in favor of Ryan's node coloring technique
#     z_order = write_utils.morton_order_cube(cube_side=4)

node_assignments = write_utils.node_assignment(cube_side=4)

In [16]:
range_list[:15]

[[[0, 512], [0, 512], [0, 512]],
 [[0, 512], [0, 512], [512, 1024]],
 [[0, 512], [0, 512], [1024, 1536]],
 [[0, 512], [0, 512], [1536, 2048]],
 [[0, 512], [512, 1024], [0, 512]],
 [[0, 512], [512, 1024], [512, 1024]],
 [[0, 512], [512, 1024], [1024, 1536]],
 [[0, 512], [512, 1024], [1536, 2048]],
 [[0, 512], [1024, 1536], [0, 512]],
 [[0, 512], [1024, 1536], [512, 1024]],
 [[0, 512], [1024, 1536], [1024, 1536]],
 [[0, 512], [1024, 1536], [1536, 2048]],
 [[0, 512], [1536, 2048], [0, 512]],
 [[0, 512], [1536, 2048], [512, 1024]],
 [[0, 512], [1536, 2048], [1024, 1536]]]

In [17]:
folders=write_utils.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/")

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_" + write_type + "/"

# folders[:5]

# Create top-level dirs

# for folder_path in folders:
#     os.makedirs(folder_path, exist_ok=False)

In [18]:
sorted_morton_list = [] # Sorting by Morton code to be consistent with Isotropic8192
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    sorted_morton_list.append((write_utils.morton_pack(array_cube_side, min_coord[0], min_coord[1], min_coord[2]), write_utils.morton_pack(array_cube_side, max_coord[0], max_coord[1], max_coord[2])))
        # (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))

        
sorted_morton_list = sorted(sorted_morton_list)
sorted_morton_list[:5]

[(0, 134217727),
 (134217728, 268435455),
 (268435456, 402653183),
 (402653184, 536870911),
 (536870912, 671088639)]

In [19]:
mike_dict = {}
for i in range(len(range_list)):            
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
            
    mike_dict[dest_folder_name + str(i + 1).zfill(2)] = sorted_morton_list[i]
#         (morton_pack(array_cube_side, min_coord[0], min_coord[1], min_coord[2]), morton_pack(array_cube_side, max_coord[0], max_coord[1], max_coord[2]))
        # (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
        
    # Ordered 2-1-0 bcs. dims are nnz-nny-nnx
    
#     print(mike_dict[dest_folder_name + str(i + 1).zfill(2)])

In [20]:
mike_dict

{'sabl2048b01': (0, 134217727),
 'sabl2048b02': (134217728, 268435455),
 'sabl2048b03': (268435456, 402653183),
 'sabl2048b04': (402653184, 536870911),
 'sabl2048b05': (536870912, 671088639),
 'sabl2048b06': (671088640, 805306367),
 'sabl2048b07': (805306368, 939524095),
 'sabl2048b08': (939524096, 1073741823),
 'sabl2048b09': (1073741824, 1207959551),
 'sabl2048b10': (1207959552, 1342177279),
 'sabl2048b11': (1342177280, 1476395007),
 'sabl2048b12': (1476395008, 1610612735),
 'sabl2048b13': (1610612736, 1744830463),
 'sabl2048b14': (1744830464, 1879048191),
 'sabl2048b15': (1879048192, 2013265919),
 'sabl2048b16': (2013265920, 2147483647),
 'sabl2048b17': (2147483648, 2281701375),
 'sabl2048b18': (2281701376, 2415919103),
 'sabl2048b19': (2415919104, 2550136831),
 'sabl2048b20': (2550136832, 2684354559),
 'sabl2048b21': (2684354560, 2818572287),
 'sabl2048b22': (2818572288, 2952790015),
 'sabl2048b23': (2952790016, 3087007743),
 'sabl2048b24': (3087007744, 3221225471),
 'sabl2048b25':

In [21]:
def flatten_3d_list(lst_3d):
    return [element for sublist_2d in lst_3d for sublist_1d in sublist_2d for element in sublist_1d]

def search_dict_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None  # Value not found in the dictionary

In [22]:
flattened_node_assgn = flatten_3d_list(node_assignments)
print(flattened_node_assgn)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [23]:
len(range_list)

64

In [24]:
cubes = range_list

In [25]:
for i in range(len(range_list)):
    min_coord = [a[0] for a in range_list[i]]
    max_coord = [a[1] - 1 for a in range_list[i]]
    
    # nnz - nny - nnx
    morton = (write_utils.morton_pack(array_cube_side, min_coord[2], min_coord[1], min_coord[0]), write_utils.morton_pack(array_cube_side, max_coord[2], max_coord[1], max_coord[0]))
    
    chunk_name = search_dict_by_value(mike_dict, morton)
    
    idx = int(chunk_name[-2:].lstrip('0'))
    
    filedb_index = flattened_node_assgn[idx - 1] - 1
    
    destination = os.path.join(folders[filedb_index], dest_folder_name + str(idx).zfill(2) + "_" + str(timestep_nr).zfill(3) + ".zarr")
    print(destination)
    
    current_array = cubes[i]
    print(current_array, '|' , morton)

/home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_001.zarr
[[0, 512], [0, 512], [0, 512]] | (0, 134217727)
/home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_001.zarr
[[0, 512], [0, 512], [512, 1024]] | (134217728, 268435455)
/home/idies/workspace/turb/data09_01/zarr/sabl2048b_09_prod/sabl2048b09_001.zarr
[[0, 512], [0, 512], [1024, 1536]] | (1073741824, 1207959551)
/home/idies/workspace/turb/data10_01/zarr/sabl2048b_10_prod/sabl2048b10_001.zarr
[[0, 512], [0, 512], [1536, 2048]] | (1207959552, 1342177279)
/home/idies/workspace/turb/data03_01/zarr/sabl2048b_03_prod/sabl2048b03_001.zarr
[[0, 512], [512, 1024], [0, 512]] | (268435456, 402653183)
/home/idies/workspace/turb/data04_01/zarr/sabl2048b_04_prod/sabl2048b04_001.zarr
[[0, 512], [512, 1024], [512, 1024]] | (402653184, 536870911)
/home/idies/workspace/turb/data11_01/zarr/sabl2048b_11_prod/sabl2048b11_001.zarr
[[0, 512], [512, 1024], [1024, 1536]] | (1342177280, 1476395007)
/home/idies/work

In [26]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [27]:
import dill

pickle_file = './filename_morton_ranges.pickle'

# the dictionary that will be written to the pickle file. empty dictionary in this example, but you would fill it with the data.
filename_morton_map = mike_dict

with open(pickle_file, 'wb') as pickled_filepath:
    dill.dump(filename_morton_map, pickled_filepath)